In [1]:
%matplotlib inline
import os
import nltk
import re
import numpy as np
import regex
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import pandas as pd
import string

In [2]:
loose=pd.read_csv('original-sheets\Responses-Loose.csv')
student_info=pd.read_csv('corpus-files\student_information.csv')
course=pd.read_csv('corpus-files\course.csv')
answer_df=pd.read_csv('answer.csv',index_col = 'answer_id', low_memory=False)


In [3]:
#dropping to only relevant columns
loose=loose.drop(columns=['3Langs','Item','gender','course_id','class_id','version','text1_len','text2_len','text1',
                         'text2 (line breaks/extra spaces removed, spaces added to reach 60)','Judgement','Notes'])

In [4]:
course_=course.drop(columns=['class_id','semester','section'])

student=student_info.drop(columns=['birth_year','language_used_at_home','non_native_language_1','yrs_of_study_lang1',
                                  'study_in_classroom_lang1','ways_of_study_lang1','non_native_language_2','study_in_classroom_lang2',
                                  'ways_of_study_lang2','non_native_language_3','yrs_of_study_lang3','study_in_classroom_lang3',
                                  'ways_of_study_lang3','yrs_of_english_learning','yrs_in_english_environment','yrs_of_study_lang2'])
answer_=answer_df[['question_id','anon_id','course_id','version','text_len','text','tokens']].reset_index()

In [5]:
#This function removes all of the rows that contain a string in the passed column
def rmStr (df, col):
    row_index=0
    row_ind=[]
    for i in df[col]:
        try:
            int(i)
        except:
            row_ind.append(row_index)
        row_index+=1
    return df.drop(labels=row_ind,axis=0)

In [6]:
#Cleaning the answer df
col='course_id'
answer=rmStr(answer_, col) #removes strings from answer_.course_id 

In [7]:
merge_ans=answer[['anon_id','course_id']] #sets answer df up for merging 
student_merge=student.drop(columns=['course_history']) #set student df up for merging

student_ans=student_merge.merge(answer,on='anon_id').astype({'course_id':'int64'}) #merges student and answers

stu_ans_crs_=student_ans.merge(course_, on='course_id').drop(columns=['age','course_id','version','tokens']) #merges the student-answers df with course df

In [8]:
# function that finds the unique values in a list
def get_uniques(x):
    return list(x.unique())

# Whole Dataset

In [9]:
# adding spaces after punctuation if needed

index=0
stu_ans_crs=stu_ans_crs_.copy()
stu_ans_crs['processed_text']=pd.NaT
stu_ans_crs['wordtype_len']=pd.NaT
words=[]
for text in stu_ans_crs.text:
    uwords=[]
    text=text.replace('\n',' ')
    nopunc=text.translate(str.maketrans('','',string.punctuation))
    nopunc=nopunc.lower()
    stu_ans_crs.at[index,'processed_text']=nopunc
    words=nopunc.strip().split(" ")
    for word in words:
        if word=='':
            words.remove(word)
        elif word not in uwords:
            uwords.append(word)
    stu_ans_crs.at[index,'text_len']=len(words)
    stu_ans_crs.at[index,'wordtype_len']=len(uwords)
    index+=1

In [10]:
#dropping all rows where the answer is less than 60 words
index=0
ind_list=[]
for i in stu_ans_crs.text_len:
    txtlen=int(i)
    if txtlen < 60:
        ind_list.append(index)
    index+=1

stu_ans_crs=stu_ans_crs.drop(labels=ind_list,axis=0).reset_index()

In [11]:
sac_lvls=stu_ans_crs[['anon_id','native_language','level_id','question_id','text']]
sac_lvls_rkc=sac_lvls

index=0
for i in sac_lvls_rkc.native_language:
    if (i != 'Korean' and i != 'Spanish' and i != 'Chinese'):
        sac_lvls_rkc=sac_lvls_rkc.drop(index)
    index+=1

sac_lvls_rkc=sac_lvls_rkc.reset_index()

sac_lvls_rkc

,index,anon_id,native_language,level_id,question_id,text
0,13,ax4,Chinese,5,48,The Effects on Students Who Have a Part-time J...
1,14,ax4,Chinese,5,97,There are many different kinds of people in th...
2,15,ax4,Chinese,5,74,Should the Death Penalty Exist\nThe death pena...
3,16,ax4,Chinese,5,119,Should the Death Penalty Exist?\nThe death pen...
4,17,ax4,Chinese,5,190,The nationality rooms built in the Cathedral o...
...,...,...,...,...,...,...
7575,17135,ew6,Chinese,3,6087,I participated student organization in univers...
7576,17136,ew6,Chinese,3,6119,My ideal home is big and modern. It is located...
7577,17146,cz3,Korean,3,6027,I will put 5 items in time capsule such as som...
7578,17147,cz3,Korean,3,6087,I have studied Visual Design since 2002. I gra...


In [12]:
# allows us to see what levels each anon_id participated in 
lvl_list=sac_lvls_rkc.groupby('anon_id').agg(n_uniq=('question_id','nunique'), lvl_nums=('question_id',get_uniques))
lvl_list=lvl_list.reset_index()

In [13]:
unique_q=sac_lvls_rkc.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

In [14]:
unique_q

,native_language,n_uniq
0,Chinese,1442
1,Korean,1353
2,Spanish,595


In [15]:
unique_t=sac_lvls_rkc.groupby('native_language').agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()
unique_t

,native_language,n_uniq
0,Chinese,3180
1,Korean,3522
2,Spanish,772


In [16]:
unique_a=sac_lvls_rkc.groupby('native_language').agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()
unique_a

,native_language,n_uniq
0,Chinese,210
1,Korean,208
2,Spanish,57


In [17]:
sac_lvls_rkc

,index,anon_id,native_language,level_id,question_id,text
0,13,ax4,Chinese,5,48,The Effects on Students Who Have a Part-time J...
1,14,ax4,Chinese,5,97,There are many different kinds of people in th...
2,15,ax4,Chinese,5,74,Should the Death Penalty Exist\nThe death pena...
3,16,ax4,Chinese,5,119,Should the Death Penalty Exist?\nThe death pen...
4,17,ax4,Chinese,5,190,The nationality rooms built in the Cathedral o...
...,...,...,...,...,...,...
7575,17135,ew6,Chinese,3,6087,I participated student organization in univers...
7576,17136,ew6,Chinese,3,6119,My ideal home is big and modern. It is located...
7577,17146,cz3,Korean,3,6027,I will put 5 items in time capsule such as som...
7578,17147,cz3,Korean,3,6087,I have studied Visual Design since 2002. I gra...


In [18]:
#dividing my level 3 and language
sac_lvls_rkc3=sac_lvls_rkc

index=0
for i in sac_lvls_rkc3.level_id:
    if i!=3:
        sac_lvls_rkc3=sac_lvls_rkc3.drop(index)
    index+=1
    
sac_lvls_rkc3

,index,anon_id,native_language,level_id,question_id,text
445,1002,fk5,Korean,3,31,Here are a few instructions for typing in Engl...
446,1003,fk5,Korean,3,98,"Personal Letter\n \n October 16, 2006 \n\nDear..."
447,1004,fk5,Korean,3,98,"Personal Letter\n \n October 20, 2006 \nDear B..."
448,1005,fk5,Korean,3,121,"Personal Letter\n \n October 20, 2006 \nDear B..."
449,1006,fk5,Korean,3,159,Success\n \n\n On of the things that I have be...
...,...,...,...,...,...,...
7575,17135,ew6,Chinese,3,6087,I participated student organization in univers...
7576,17136,ew6,Chinese,3,6119,My ideal home is big and modern. It is located...
7577,17146,cz3,Korean,3,6027,I will put 5 items in time capsule such as som...
7578,17147,cz3,Korean,3,6087,I have studied Visual Design since 2002. I gra...


In [19]:
unique_q=sac_lvls_rkc3.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

unique_t=sac_lvls_rkc3.groupby('native_language').agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()

unique_a=sac_lvls_rkc3.groupby('native_language').agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()

In [20]:
unique_q

,native_language,n_uniq
0,Chinese,262
1,Korean,259
2,Spanish,119


In [21]:
unique_t

,native_language,n_uniq
0,Chinese,531
1,Korean,634
2,Spanish,162


In [22]:
unique_a

,native_language,n_uniq
0,Chinese,74
1,Korean,81
2,Spanish,17


In [23]:
#dividing by level 4 and language
sac_lvls_rkc4=sac_lvls_rkc

index=0
for i in sac_lvls_rkc4.level_id:
    if i!=4:
        sac_lvls_rkc4=sac_lvls_rkc4.drop(index)
    index+=1
    
sac_lvls_rkc4

,index,anon_id,native_language,level_id,question_id,text
71,184,cd6,Chinese,4,116,1)Creating mind movies.\n2)Setting the right g...
72,185,cd6,Chinese,4,156,reserve (adj.)\nDefinition: an amount of somet...
73,187,gq4,Spanish,4,116,Five pieces of advice from the reading:\n1. Pr...
74,188,gq4,Spanish,4,116,Five pieces of advice from the reading:\n1. Pr...
75,189,gq4,Spanish,4,156,1. Folks (Title): noun\nGroup of people\n\n2. ...
...,...,...,...,...,...,...
7543,16973,ff6,Chinese,4,6030,"If I could choose, I would not like to live in..."
7544,16974,ff6,Chinese,4,6117,I consider that the most effective way to mark...
7545,16979,ev5,Chinese,4,6002,Hikikomori lifestyle is the most unusual of al...
7546,16980,ev5,Chinese,4,6030,"I don't like to live in the leisureville, beca..."


In [24]:
unique_q=sac_lvls_rkc4.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

unique_t=sac_lvls_rkc4.groupby('native_language').agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()

unique_a=sac_lvls_rkc4.groupby('native_language').agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()

In [25]:
unique_t

,native_language,n_uniq
0,Chinese,1507
1,Korean,1711
2,Spanish,369


In [26]:
unique_a

,native_language,n_uniq
0,Chinese,148
1,Korean,150
2,Spanish,40


In [27]:
#dividing my level 5 and language
sac_lvls_rkc5=sac_lvls_rkc

index=0
for i in sac_lvls_rkc5.level_id:
    if i!=5:
        sac_lvls_rkc5=sac_lvls_rkc5.drop(index)
    index+=1
    
sac_lvls_rkc5

,index,anon_id,native_language,level_id,question_id,text
0,13,ax4,Chinese,5,48,The Effects on Students Who Have a Part-time J...
1,14,ax4,Chinese,5,97,There are many different kinds of people in th...
2,15,ax4,Chinese,5,74,Should the Death Penalty Exist\nThe death pena...
3,16,ax4,Chinese,5,119,Should the Death Penalty Exist?\nThe death pen...
4,17,ax4,Chinese,5,190,The nationality rooms built in the Cathedral o...
...,...,...,...,...,...,...
7465,16697,bf7,Chinese,5,6136,"Recently, I have watched a comedy starring Mar..."
7466,16722,go7,Chinese,5,6000,I am confused by the East Island status. It i...
7467,16723,go7,Chinese,5,6043,"When I was in class at lunch time today, I hav..."
7468,16724,go7,Chinese,5,6104,Today I have listened a part of ANON_NAME_0 S...


In [28]:
unique_q=sac_lvls_rkc5.groupby('native_language').agg(n_uniq=('question_id','nunique'))
unique_q=unique_q.reset_index()

unique_t=sac_lvls_rkc5.groupby('native_language').agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()

unique_a=sac_lvls_rkc5.groupby('native_language').agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()

In [29]:
unique_t

,native_language,n_uniq
0,Chinese,1121
1,Korean,1160
2,Spanish,238


In [30]:
unique_a

,native_language,n_uniq
0,Chinese,95
1,Korean,109
2,Spanish,25


# Finding texts across all levels Whole

In [35]:
ind=0
ss=sac_lvls_rkc.reset_index()
for i in ss.anon_id:
    if ss.at[ind,'level_id']==2:
        ss=ss.drop(ind)
    ind+=1
ss=ss.drop(columns=['level_0'])
ss=ss.reset_index()
ss

,level_0,index,anon_id,native_language,level_id,question_id,text
0,0,13,ax4,Chinese,5,48,The Effects on Students Who Have a Part-time J...
1,1,14,ax4,Chinese,5,97,There are many different kinds of people in th...
2,2,15,ax4,Chinese,5,74,Should the Death Penalty Exist\nThe death pena...
3,3,16,ax4,Chinese,5,119,Should the Death Penalty Exist?\nThe death pen...
4,4,17,ax4,Chinese,5,190,The nationality rooms built in the Cathedral o...
...,...,...,...,...,...,...,...
7534,7534,17135,ew6,Chinese,3,6087,I participated student organization in univers...
7535,7535,17136,ew6,Chinese,3,6119,My ideal home is big and modern. It is located...
7536,7536,17146,cz3,Korean,3,6027,I will put 5 items in time capsule such as som...
7537,7537,17147,cz3,Korean,3,6087,I have studied Visual Design since 2002. I gra...


In [37]:
 # this code gets how many times an id appeared (n_anon_ids) as well as an array of what levels they participated in (lvl_nums)
slr=sac_lvls_rkc.groupby('anon_id').agg(
    n_anon_ids_whole=('level_id', 'nunique'),
    lvl_nums_whole=('level_id', get_uniques)
)

#changes the clean_ids_lvl's index so we can access the anon_id easier
ind_whole= slr.reset_index()
ind_whole

,anon_id,n_anon_ids_whole,lvl_nums_whole
0,aa0,1,[5]
1,aa1,1,[4]
2,aa3,1,[4]
3,aa8,2,"[5, 4]"
4,aa9,2,"[3, 4]"
...,...,...,...
468,ha0,1,[5]
469,ha2,2,"[5, 4]"
470,ha6,2,"[3, 4]"
471,hb4,3,"[5, 4, 3]"


In [38]:
# for loop that checks if lvl_nums has all three levels in it (3,4, and 5)
# if there are three values in lvl_nums, the anon_id is appended to the all_three list
ind=0
wall_three_loose=[]
append=''
for i in ind_whole.anon_id:
    if ind_whole.iat[ind, 1]==3:
        append=ind_whole.at[ind,'anon_id']
        wall_three_loose.append(append)
    ind=ind+1

wall_three_loose.sort()
len(wall_three_loose)

42

In [39]:
# creates list of participants in loose sheet that are in level 3 and 4
ind=0
in34_whole=[]
append=''
for i in ind_whole.anon_id:
    if ind_whole.iat[ind, 1]==2:
        if (3 in ind_whole.iat[ind,2] and 4 in ind_whole.iat[ind,2]):
            append=ind_whole.at[ind,'anon_id']
            in34_whole.append(append)
    elif ind_whole.iat[ind,1]==3:
        append=ind_whole.at[ind,'anon_id']
        in34_whole.append(append)
    ind=ind+1
in34_whole.sort()
print(len(in34_whole))

111


In [40]:
# creates list of participants in loose sheet that are in level 3 and 4
ind=0
in45_whole=[]
append=''
for i in ind_whole.anon_id:
    if ind_whole.iat[ind, 1]==2:
        if (4 in ind_whole.iat[ind,2] and 5 in ind_whole.iat[ind,2]):
            append=ind_whole.at[ind,'anon_id']
            in45_whole.append(append)
    elif ind_whole.iat[ind,1]==3:
        append=ind_whole.at[ind,'anon_id']
        in45_whole.append(append)
    ind=ind+1
in45_whole.sort()
print(len(in45_whole))

154


In [42]:
wall_three=ss
index=0
for i in wall_three.anon_id:
    if i not in wall_three_loose:
        wall_three=wall_three.drop(index)
    index+=1
    
#wall_three=wall_three.reset_index()
wall_three

,level_0,index,anon_id,native_language,level_id,question_id,text
143,143,313,ev9,Chinese,4,2220,Dr. Schwartz's advices are very crucial and u...
144,144,314,ev9,Chinese,4,2282,There are two reasons that I think Martin is m...
145,145,315,ev9,Chinese,4,2493,"If I am going to start a new business, I will..."
148,148,318,co5,Korean,4,2220,She suggested several tips of online dating. ...
149,149,319,co5,Korean,4,2282,I am convinced that Martin really has psychic...
...,...,...,...,...,...,...,...
7407,7407,16644,bf7,Chinese,5,5995,Because of liquor could be had easily at a loc...
7408,7408,16645,bf7,Chinese,5,6040,"1. In Taiwan's middle school system, there are..."
7422,7422,16695,bf7,Chinese,5,6000,I am fascinated by mysteries. Mysteries are in...
7423,7423,16696,bf7,Chinese,5,6043,"No matter my present or my future life, there ..."


In [43]:
whole34=ss
index=0
for i in whole34.anon_id:
    if i not in in34_whole:
        whole34=whole34.drop(index)
    index+=1

whole34

,level_0,index,anon_id,native_language,level_id,question_id,text
140,140,310,ca6,Korean,4,2220,I really impressed when I read Dr.Schwartz's ...
141,141,311,ca6,Korean,4,2282,"In my opinion, I don't believe unexplainable m..."
142,142,312,ca6,Korean,4,2493,"If I'm going to start a business, I'll use th..."
143,143,313,ev9,Chinese,4,2220,Dr. Schwartz's advices are very crucial and u...
144,144,314,ev9,Chinese,4,2282,There are two reasons that I think Martin is m...
...,...,...,...,...,...,...,...
7505,7505,16980,ev5,Chinese,4,6030,"I don't like to live in the leisureville, beca..."
7506,7506,16981,ev5,Chinese,4,6117,"I think, different effective way for different..."
7507,7507,17007,ff6,Chinese,3,5763,A day on a beach\nSee hear smell taste feel to...
7508,7508,17008,ff6,Chinese,3,5763,A day at the beach\nIt was a sunny day. We wen...


In [44]:
whole45=ss
index=0
for i in whole45.anon_id:
    if i not in in45_whole:
        whole45=whole45.drop(index)
    index+=1
    
whole45

,level_0,index,anon_id,native_language,level_id,question_id,text
5,5,18,fg8,Chinese,5,4,"In Taiwan, we have a proverb, ""Far relative ca..."
6,6,19,fg8,Chinese,5,48,The Effects of Watching movies\n\nWith the dev...
7,7,20,fg8,Chinese,5,74,Making Friends on Internet\n\nWith the develop...
8,8,21,fg8,Chinese,5,97,Many people like to ask people who have good e...
9,9,22,fg8,Chinese,5,119,Making Friends on Internet\n\nWith the develop...
...,...,...,...,...,...,...,...
7480,7480,16897,bv8,Chinese,4,5912,My birthday party always cheers me up and it i...
7481,7481,16898,bv8,Chinese,4,5960,"My name is Tike, but my friends call me Tikki...."
7482,7482,16899,bv8,Chinese,4,6029,"In the whole of my life, I had experienced tha..."
7483,7483,16900,bv8,Chinese,4,6078,"2. I lived with my parents and my sibling, who..."


In [45]:
#number of unique texts
unique_t=wall_three.groupby(['native_language','level_id']).agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()
unique_t

,native_language,level_id,n_uniq
0,Chinese,3,111
1,Chinese,4,181
2,Chinese,5,157
3,Korean,3,168
4,Korean,4,339
5,Korean,5,267
6,Spanish,3,36
7,Spanish,4,52
8,Spanish,5,20


In [52]:
#total number of texts, includes duplicates
unique_t=wall_three.groupby(['native_language','level_id']).agg(text_count=('text','count'))
unique_t=unique_t.reset_index()
unique_t

,native_language,level_id,n_uniq
0,Chinese,3,112
1,Chinese,4,188
2,Chinese,5,157
3,Korean,3,171
4,Korean,4,351
5,Korean,5,273
6,Spanish,3,36
7,Spanish,4,52
8,Spanish,5,20


In [48]:
texts=wall_three.groupby(['native_language']).agg(id_count=('text','count'))
texts=texts.reset_index()
texts

,native_language,n_uniq
0,Chinese,457
1,Korean,795
2,Spanish,108


In [51]:
texts=wall_three.groupby(['native_language']).agg(n_uniq=('anon_id','count'))
texts

,n_uniq
native_language,
Chinese,457
Korean,795
Spanish,108


In [59]:
#total number of texts, includes duplicates
whole43=whole34.query('level_id!=5')
unique_t=whole43.groupby(['native_language','level_id']).agg(text_count=('text','count'))
unique_t=unique_t.reset_index()
unique_t

,native_language,level_id,text_count
0,Chinese,3,322
1,Chinese,4,425
2,Korean,3,498
3,Korean,4,765
4,Spanish,3,67
5,Spanish,4,70


In [60]:
texts=whole43.groupby(['native_language']).agg(n_uniq=('anon_id','count'))
texts

,n_uniq
native_language,
Chinese,747
Korean,1263
Spanish,137


In [61]:
part=whole43.groupby(['native_language']).agg(n_uniq=('anon_id','nunique'))
part

,n_uniq
native_language,
Chinese,42
Korean,60
Spanish,9


In [62]:
#total number of texts, includes duplicates
whole54=whole45.query('level_id!=3')
unique_t=whole54.groupby(['native_language','level_id']).agg(text_count=('text','count'))
unique_t=unique_t.reset_index()
unique_t

,native_language,level_id,text_count
0,Chinese,4,781
1,Chinese,5,805
2,Korean,4,983
3,Korean,5,820
4,Spanish,4,143
5,Spanish,5,119


In [63]:
texts=whole54.groupby(['native_language']).agg(n_uniq=('anon_id','count'))
texts

,n_uniq
native_language,
Chinese,1586
Korean,1803
Spanish,262


In [64]:
part=whole54.groupby(['native_language']).agg(n_uniq=('anon_id','nunique'))
part

,n_uniq
native_language,
Chinese,66
Korean,72
Spanish,16


In [65]:
ss

,level_0,index,anon_id,native_language,level_id,question_id,text
0,0,13,ax4,Chinese,5,48,The Effects on Students Who Have a Part-time J...
1,1,14,ax4,Chinese,5,97,There are many different kinds of people in th...
2,2,15,ax4,Chinese,5,74,Should the Death Penalty Exist\nThe death pena...
3,3,16,ax4,Chinese,5,119,Should the Death Penalty Exist?\nThe death pen...
4,4,17,ax4,Chinese,5,190,The nationality rooms built in the Cathedral o...
...,...,...,...,...,...,...,...
7534,7534,17135,ew6,Chinese,3,6087,I participated student organization in univers...
7535,7535,17136,ew6,Chinese,3,6119,My ideal home is big and modern. It is located...
7536,7536,17146,cz3,Korean,3,6027,I will put 5 items in time capsule such as som...
7537,7537,17147,cz3,Korean,3,6087,I have studied Visual Design since 2002. I gra...


In [68]:
partlvl=ss.groupby(['native_language','level_id']).agg(part=('anon_id','nunique'))
partlvl

part
native_language level_id      
Chinese         3           74
                4          148
                5           95
Korean          3           81
                4          150
                5          109
Spanish         3           17
                4           40
                5           25

In [71]:
probes=ss.groupby(['native_language']).agg(probes=('question_id','count'))
probes

,probes
native_language,
Chinese,3204
Korean,3555
Spanish,780


In [72]:
probes=ss.groupby(['native_language']).agg(probes=('question_id','nunique'))
probes

,probes
native_language,
Chinese,1428
Korean,1346
Spanish,592


In [76]:
text=ss.groupby(['native_language']).agg(texts=('text','count'))
text

,texts
native_language,
Chinese,3204
Korean,3555
Spanish,780


In [77]:
part=ss.groupby(['native_language']).agg(participants=('anon_id','nunique'))
part

,participants
native_language,
Chinese,208
Korean,208
Spanish,57


# Loose Dataset

In [ ]:
loose=loose.dropna()
loose

In [ ]:
    # this code gets how many times an id appeared (n_anon_ids) as well as an array of what levels they participated in (lvl_nums)
clean_ids_lvl_loose=loose.groupby('anon_id').agg(
    n_anon_ids_loose=('level_id', 'nunique'),
    lvl_nums_loose=('level_id', get_uniques)
)

#changes the clean_ids_lvl's index so we can access the anon_id easier
ind_loose= clean_ids_lvl_loose.reset_index()
ind_loose

In [ ]:
# for loop that checks if lvl_nums has all three levels in it (3,4, and 5)
# if there are three values in lvl_nums, the anon_id is appended to the all_three list
ind=0
all_three_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==3:
        append=ind_loose.at[ind,'anon_id']
        all_three_loose.append(append)
    ind=ind+1

all_three_loose.sort()
len(all_three_loose)

In [ ]:
# creates list of participants in loose sheet that are in level 3 and 4
ind=0
in34_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==2:
        if (3 in ind_loose.iat[ind,2] and 4 in ind_loose.iat[ind,2]):
            append=ind_loose.at[ind,'anon_id']
            in34_loose.append(append)
    elif ind_loose.iat[ind,1]==3:
        append=ind_loose.at[ind,'anon_id']
        in34_loose.append(append)
    ind=ind+1
in34_loose.sort()
print(len(in34_loose))

In [ ]:
# creates list of participants in loose sheet that are in level 4 and 5
ind=0
in45_loose=[]
append=''
for i in ind_loose.anon_id:
    if ind_loose.iat[ind, 1]==2:
        if (4 in ind_loose.iat[ind,2] and 5 in ind_loose.iat[ind,2]):
            append=ind_loose.at[ind,'anon_id']
            in45_loose.append(append)
    elif ind_loose.iat[ind,1]==3:
        append=ind_loose.at[ind,'anon_id']
        in45_loose.append(append)
    ind=ind+1
in45_loose.sort()
print(len(in45_loose))

# Finding number of texts breakdown

In [ ]:
index=0
textcount_loose=loose.copy()

textcount_loose['processed_text']=pd.NaT
textcount_loose['wordtype_len']=pd.NaT
words=[]
for text in textcount_loose['text3 (edits made to fix word counts)']:
    uwords=[]
    text=str(text).replace('\n',' ')
    nopunc=text.translate(str.maketrans('','',string.punctuation))
    nopunc=nopunc.lower()
    textcount_loose.at[index,'processed_text']=nopunc
    words=nopunc.strip().split(" ")
    for word in words:
        if word=='':
            words.remove(word)
        elif word not in uwords:
            uwords.append(word)
    textcount_loose.at[index,'text3_len']=len(words)
    textcount_loose.at[index,'wordtype_len']=len(uwords)
    index+=1

In [ ]:
all_three_loose

In [ ]:
all_three=loose
index=0
for i in all_three.anon_id:
    if i not in all_three_loose:
        all_three=all_three.drop(index)
    index+=1
    
all_three=all_three.reset_index()
all_three

In [ ]:
loose34=loose
index=0
for i in loose34.anon_id:
    if i not in in34_loose:
        loose34=loose34.drop(index)
    index+=1
    
loose34=loose34.reset_index()
loose34

In [ ]:
loose45=loose
index=0
for i in loose45.anon_id:
    if i not in in45_loose:
        loose45=loose45.drop(index)
    index+=1
    
loose45=loose45.reset_index()
loose45

In [ ]:
unique_t=all_three.groupby(['L1','level_id']).agg(n_uniq=('text3 (edits made to fix word counts)','nunique'))
unique_t=unique_t.reset_index()
unique_t

In [ ]:
unique_t=loose34.groupby(['L1','level_id']).agg(n_uniq=('text3 (edits made to fix word counts)','nunique'))
unique_t=unique_t.reset_index()
unique_t

In [ ]:
unique_t=loose45.groupby(['L1','level_id']).agg(n_uniq=('text3 (edits made to fix word counts)','nunique'))
unique_t=unique_t.reset_index()
unique_t

In [ ]:
whole3=sac_lvls_rkc.query('level_id==3')

In [ ]:
# function that counts the amount of tokens per language FINDS AVERAGE OF ALL LVLS IN SPECIFIED LIST
def finding_tokens(df,lang_col,text_len_col,qcounts):
    spanish=0
    korean=0
    chinese=0
    index=0
    lspanish=0
    lchinese=0
    lkorean=0
    
    for i in df['anon_id']:
        lang=df.at[index,lang_col]
        length=int(df.at[index,text_len_col])
        if lang=='Spanish':
            lspanish+=1
            spanish+=length
        elif lang=='Korean':
            lkorean+=1
            korean+=length
        elif lang=='Chinese':
            lchinese+=1
            chinese+=length
        index+=1
   # print('Spanish Token Average: ',spanish,'\nKorean Token Average: ',korean,
    #      '\nChinese Token Average:',chinese)
    print('Spanish Token Average: ',spanish/lspanish,'\nKorean Token Average: ',korean/lkorean,
          '\nChinese Token Average:',chinese/lchinese)

In [ ]:
#finds the token average of the given level in the given level group (id_list) for each language
def typeAvg(df,typecol,lang, lvl):
    index=0
    #total token count
    spanish=0
    chinese=0
    korean=0
    #number of questions
    qcounts=0
    qcountc=0
    qcountk=0

    for anon in df['anon_id']:
        if df.at[index,'level_id']==lvl: # and df.at[index,'anon_id'] in id_list:
            textlen=df.at[index,typecol]
            textlen=int(textlen)
            if df.at[index,lang]=='Spanish':
                spanish+=textlen
                qcounts+=1
            elif df.at[index,lang]=='Korean':
                korean+=textlen
                qcountk+=1
            elif df.at[index,lang]=='Chinese':
                chinese+=textlen
                qcountc+=1
        index+=1
        
    print("Spanish Average: ",spanish/qcounts)
    print("Korean Average: ",korean/qcountk)
    print("Chinese Average: ",chinese/qcountc)

In [ ]:
typeAvg(textcount_loose,'text3_len','L1',3)

In [ ]:
typeAvg(textcount_loose,'text3_len','L1',4)

In [ ]:
typeAvg(textcount_loose,'text3_len','L1',5)

In [ ]:
typeAvg(textcount_loose, 'wordtype_len', 'L1', 3)

In [ ]:
typeAvg(textcount_loose, 'wordtype_len', 'L1', 4)

In [ ]:
typeAvg(textcount_loose, 'wordtype_len', 'L1', 5)

In [ ]:
#finding the token/word count ratio for each language
def ratio(df,textlencol,typelencol,lang,lvl):
    index=0
    #total token count
    spanish=0
    chinese=0
    korean=0
    #unique token count
    tspanish=0
    tkorean=0
    tchinese=0
    #total question count
    qcounts=0
    qcountc=0
    qcountk=0

    for anon in df['anon_id']:
        if df.at[index,'level_id']==lvl:
            textlen=df.at[index,textlencol]
            textlen=int(textlen)
            typelen=df.at[index,typelencol]
            typelen=int(typelen)
            if df.at[index,lang]=='Spanish':
                spanish+=textlen
                tspanish+=typelen
                qcounts+=1
            elif df.at[index,lang]=='Korean':
                korean+=textlen
                tkorean+=typelen
                qcountk+=1
            elif df.at[index,lang]=='Chinese':
                chinese+=textlen
                tchinese+=typelen
                qcountc+=1
        index+=1

    #finding ratios    
    spanishavg=spanish/qcounts
    tspanishavg=tspanish/qcounts
    koreanavg=korean/qcountk
    tkoreanavg=tkorean/qcountk
    chineseavg=chinese/qcountc
    tchineseavg=tchinese/qcountc
    
    print("Spanish Ratio: ",tspanishavg/spanishavg)
    print("Korean Ratio: ",tkoreanavg/koreanavg)
    print("Chinese Ratio: ",tchineseavg/chineseavg)

In [ ]:
ratio(textcount_loose,'text3_len','wordtype_len','L1',3)

In [ ]:
ratio(textcount_loose,'text3_len','wordtype_len','L1',4)

In [ ]:
ratio(textcount_loose,'text3_len','wordtype_len','L1',5)

# Whole dataset over 3-5

In [ ]:
whole=sac_lvls_rkc.query('level_id==3 or level_id==4 or level_id==5').drop(columns=['level_0'])
whole

In [ ]:
# all participants, not just 3-5
unique_t=whole.groupby(['native_language','level_id']).agg(n_uniq=('text','nunique'))
unique_t=unique_t.reset_index()
unique_t

In [ ]:
# this code gets how many times an id appeared (n_anon_ids) as well as an array of what levels they participated in (lvl_nums)
whole_id=whole.groupby('anon_id').agg(
    n_anon_ids_whole=('level_id', 'nunique'),
    lvl_nums_whole=('level_id', get_uniques)
)

#changes the clean_ids_lvl's index so we can access the anon_id easier
ind_whole= whole_id.reset_index()
ind_whole

In [ ]:
# for loop that checks if lvl_nums has all three levels in it (3,4, and 5)
# if there are three values in lvl_nums, the anon_id is appended to the all_three list
ind=0
all_three_whole=[]
append=''
for i in ind_whole.anon_id:
    if ind_whole.iat[ind, 1]==3:
        append=ind_whole.at[ind,'anon_id']
        all_three_whole.append(append)
    ind=ind+1

all_three_whole.sort()
len(all_three_whole)

In [ ]:
whole

In [ ]:
all_three=whole
index=0
for i in all_three.anon_id:
    if i not in all_three_whole:
        all_three=all_three.drop(index)
    index+=1
    
#all_three.drop(columns=['level_0','index'], inplace=True)
all_three=all_three.reset_index()

In [ ]:
print(all_three_whole)

In [ ]:
# checking
all_three.query('anon_id=="fi1" and level_id==3')

In [ ]:
all_three.drop(columns=['level_0','index'],inplace=True)
all_three

In [ ]:
unique_a=all_three.groupby(['native_language']).agg(n_uniq=('anon_id','nunique'))
unique_a=unique_a.reset_index()
unique_a

In [ ]:
unique_t=all_three.groupby(['native_language']).agg(n_uniq=('text','count'))
unique_t=unique_t.reset_index()
unique_t

In [ ]:
# all 3-5
unique_tl=all_three.groupby(['native_language','level_id']).agg(n_uniq=('text','nunique'))
unique_tl=unique_tl.reset_index()
unique_tl

In [ ]:
# all 3-5
unique_tl=all_three.groupby(['native_language','level_id']).agg(count=('text','count'))
unique_tl=unique_tl.reset_index()
unique_tl

In [ ]:
# all participants, not just 3-5
unique_tl=all_three.groupby(['level_id']).text.count()
#.agg(n_uniq=('text','nunique'))
unique_tl=unique_tl.reset_index()
unique_tl

In [ ]:
test=all_three.groupby(['text']).count()
test

In [ ]:
all_three